In [7]:
#CNN

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import label_binarize

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

#Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Data augmentation (you can customize this)
    datagen = keras.preprocessing.image.ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Feature extraction using a CNN
def create_cnn_feature_extractor(input_shape):
    model = keras.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
    ])
    return model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
feature_extractor = create_cnn_feature_extractor(input_shape)
feature_extractor.build(input_shape)
feature_extractor.compile()

# Extract features
x_train_features = feature_extractor.predict(np.array(x_train))
x_test_features = feature_extractor.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    classifier.fit(x_train_features, y_train)
    y_pred = classifier.predict(x_test_features)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

    # Calculate log loss
    if hasattr(classifier, "predict_proba"):
        y_prob = classifier.predict_proba(x_test_features)
        y_test_bin = label_binarize(y_test, classes=classifier.classes_)
        loss = log_loss(y_test_bin, y_prob)
        log_losses.append(loss)
        print(f"{name} log loss: {loss}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")



19/19 [==============================] - 1s 42ms/step
AdaBoost accuracy: 0.9116666666666666
AdaBoost log loss: 0.646517030096395
KNN accuracy: 0.9033333333333333
KNN log loss: 0.7070116376925559
RF accuracy: 0.93
RF log loss: 0.2523321375372628
SVM accuracy: 0.8933333333333333
Softmax accuracy: 0.965
Softmax log loss: 0.13604219361893874

Average Accuracy: 0.9206666666666667
Average Log Loss: 0.43547574973628816


In [2]:
#Xception


import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import label_binarize

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

#Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)
 
# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Feature extraction using Xception
def create_xception_feature_extractor(input_shape):
    base_model = keras.applications.Xception(
        include_top=False,
        weights='imagenet',
        input_shape=input_shape,
        pooling='avg'
    )
    return base_model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
feature_extractor = create_xception_feature_extractor(input_shape)

# Extract features
x_train_features = feature_extractor.predict(np.array(x_train))
x_test_features = feature_extractor.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    classifier.fit(x_train_features, y_train)
    y_pred = classifier.predict(x_test_features)
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate log loss
    if hasattr(classifier, "predict_proba"):
        y_prob = classifier.predict_proba(x_test_features)
        y_test_bin = label_binarize(y_test, classes=classifier.classes_)
        loss = log_loss(y_test_bin, y_prob)
        log_losses.append(loss)
        print(f"{name} log loss: {loss}")

    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")

19/19 [==============================] - 7s 360ms/step
AdaBoost log loss: 0.6430836191270892
AdaBoost accuracy: 0.91
KNN log loss: 0.19625383741746483
KNN accuracy: 0.9683333333333334
RF log loss: 0.2246693413830423
RF accuracy: 0.9483333333333334
SVM accuracy: 0.965
Softmax log loss: 0.05024118038015941
Softmax accuracy: 0.9816666666666667

Average Accuracy: 0.9546666666666667
Average Log Loss: 0.278561994576939


In [6]:
#InceptionV3

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import label_binarize


# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'
# Data preprocessing and augmentation
image_size = (128, 128)  # InceptionV3's required input size
batch_size = 32

#Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess data
def preprocess_data(data):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess the loaded data
    data = preprocess_data(data)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Feature extraction using InceptionV3
def create_inceptionv3_feature_extractor(input_shape):
    base_model = keras.applications.InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
feature_extractor = create_inceptionv3_feature_extractor(input_shape)

# Extract features
x_train_features = feature_extractor.predict(np.array(x_train))
x_test_features = feature_extractor.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

accuracies = []

# Train and evaluate each classifier
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    classifier.fit(x_train_features.reshape(x_train_features.shape[0], -1), y_train)
    y_pred = classifier.predict(x_test_features.reshape(x_test_features.shape[0], -1))
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate log loss
    if hasattr(classifier, "predict_proba"):
        y_prob = classifier.predict_proba(x_test_features.reshape(x_test_features.shape[0], -1))
        y_test_bin = label_binarize(y_test, classes=classifier.classes_)
        loss = log_loss(y_test_bin, y_prob)
        log_losses.append(loss)
        print(f"{name} log loss: {loss}")

    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")



19/19 [==============================] - 4s 143ms/step
AdaBoost log loss: 0.6421245399700052
AdaBoost accuracy: 0.9283333333333333
KNN log loss: 0.30897467781745463
KNN accuracy: 0.9633333333333334
RF log loss: 0.24625304437395504
RF accuracy: 0.9583333333333334
SVM accuracy: 0.9766666666666667
Softmax log loss: 0.05919000638942234
Softmax accuracy: 0.9866666666666667

Average Accuracy: 0.9626666666666667
Average Log Loss: 0.3141355671377093


In [9]:
#ResNet50


import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import label_binarize

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)  # Image size for ResNet50
batch_size = 32

#Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Resize images for ResNet50 input size
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization for ResNet50
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Feature extraction using ResNet50
preprocessed_images = np.array(data)
preprocessed_images = keras.applications.resnet50.preprocess_input(preprocessed_images)

resnet_model = keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(image_size[0], image_size[1], 3)
)

x_train_features = resnet_model.predict(np.array(x_train))
x_test_features = resnet_model.predict(np.array(x_test))

# Initialize classifiers with some parameters
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # Enable probability estimates for log loss calculation
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=2000)  # Increase max_iter
}

# Train and evaluate each classifier
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    classifier.fit(x_train_features.reshape(len(x_train), -1), y_train)
    y_pred = classifier.predict(x_test_features.reshape(len(x_test), -1))
    accuracy = accuracy_score(y_test, y_pred)
    
    # Calculate log loss
    if hasattr(classifier, "predict_proba"):
        y_prob = classifier.predict_proba(x_test_features.reshape(len(x_test), -1))
        y_test_bin = label_binarize(y_test, classes=classifier.classes_)
        loss = log_loss(y_test_bin, y_prob)
        log_losses.append(loss)
        print(f"{name} log loss: {loss}")

    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")

19/19 [==============================] - 8s 410ms/step
AdaBoost log loss: 0.6684940965888408
AdaBoost accuracy: 0.845
KNN log loss: 0.7235951727064569
KNN accuracy: 0.895
RF log loss: 0.2803952444263075
RF accuracy: 0.9166666666666666
SVM log loss: 0.4885371582376547
SVM accuracy: 0.7466666666666667
Softmax log loss: 0.2321642370077517
Softmax accuracy: 0.9483333333333334

Average Accuracy: 0.8703333333333333
Average Log Loss: 0.47863718179340226


In [11]:
#EfficientNetB0

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics import log_loss
from sklearn.preprocessing import label_binarize

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)
batch_size = 32

#Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    data = [cv2.resize(img, image_size) for img in data]
    data = np.array(data)
    data = preprocess_input(data)  # EfficientNet's preprocessing

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Define the feature extractor model
input_shape = (image_size[0], image_size[1], 3)  # 3 for RGB channels
efficientnet_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)

# Extract features using EfficientNetB0
x_train_features = efficientnet_model.predict(np.array(x_train))
x_test_features = efficientnet_model.predict(np.array(x_test))

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=2000)
}

# Train and evaluate each classifier
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    classifier.fit(x_train_features.reshape(x_train_features.shape[0], -1), y_train)
    y_pred = classifier.predict(x_test_features.reshape(x_test_features.shape[0], -1))
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

    # Calculate log loss
    if hasattr(classifier, "predict_proba"):
        y_prob = classifier.predict_proba(x_test_features.reshape(x_test_features.shape[0], -1))
        y_test_bin = label_binarize(y_test, classes=classifier.classes_)
        loss = log_loss(y_test_bin, y_prob)
        log_losses.append(loss)
        print(f"{name} log loss: {loss}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")

19/19 [==============================] - 6s 245ms/step
AdaBoost accuracy: 0.9433333333333334
AdaBoost log loss: 0.6019790878904286
KNN accuracy: 0.985
KNN log loss: 0.213250641535461
RF accuracy: 0.9716666666666667
RF log loss: 0.19731102974301248
SVM accuracy: 0.995
Softmax accuracy: 0.9983333333333333
Softmax log loss: 0.01582060232280801

Average Accuracy: 0.9786666666666667
Average Log Loss: 0.25709034037292755


In [4]:
#VGG19

import os
import numpy as np
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.preprocessing import label_binarize

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size = (128, 128)  # VGG19 input size
batch_size = 32

#Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels)

    return data, labels

# Load the dataset
data, labels = load_dataset(original_dataset_path)

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

# Preprocess for VGG19 input
x_train = np.array(x_train)
x_test = np.array(x_test)

# Normalize the images for VGG19
x_train = keras.applications.vgg19.preprocess_input(x_train)
x_test = keras.applications.vgg19.preprocess_input(x_test)

# Load the VGG19 model
vgg_model = keras.applications.VGG19(weights='imagenet', include_top=False, input_shape=image_size + (3,))

# Extract features
x_train_features = vgg_model.predict(x_train)
x_test_features = vgg_model.predict(x_test)

# Flatten extracted features
x_train_features = x_train_features.reshape(x_train_features.shape[0], -1)
x_test_features = x_test_features.reshape(x_test_features.shape[0], -1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(),
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=3000)
}

# Train and evaluate each classifier
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    classifier.fit(x_train_features, y_train)
    y_pred = classifier.predict(x_test_features)
    accuracy = accuracy_score(y_test, y_pred)
    accuracies.append(accuracy)
    print(f"{name} accuracy: {accuracy}")

    # Calculate log loss
    if hasattr(classifier, "predict_proba"):
        y_prob = classifier.predict_proba(x_test_features)
        y_test_bin = label_binarize(y_test, classes=classifier.classes_)
        loss = log_loss(y_test_bin, y_prob)
        log_losses.append(loss)
        print(f"{name} log loss: {loss}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")



19/19 [==============================] - 13s 651ms/step
AdaBoost accuracy: 0.8116666666666666
AdaBoost log loss: 0.6680632833738435
KNN accuracy: 0.88
KNN log loss: 0.9067050405190171
RF accuracy: 0.9183333333333333
RF log loss: 0.26654753044913854
SVM accuracy: 0.7116666666666667
Softmax accuracy: 0.8083333333333333
Softmax log loss: 0.4524531363136041

Average Accuracy: 0.826
Average Log Loss: 0.5734422476639008


In [1]:
#EfficientNetB0 + Xception
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0, Xception  # Change here
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size_efficientnet = (128, 128)  # EfficientNetB0's required input size
image_size_xception = (128, 128)  # Xception's required input size
batch_size = 32

# Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)

    return data, labels

# Load the EfficientNetB0 dataset
data_efficientnet, labels_efficientnet = load_dataset(original_dataset_path, image_size_efficientnet)

# Split the EfficientNetB0 data into training and testing sets
x_train_efficientnet, x_test_efficientnet, y_train_efficientnet, y_test_efficientnet = train_test_split(
    data_efficientnet, labels_efficientnet, test_size=0.2, random_state=42
)

# Load the Xception dataset
data_xception, labels_xception = load_dataset(original_dataset_path, image_size_xception)

# Split the Xception data into training and testing sets
x_train_xception, x_test_xception, y_train_xception, y_test_xception = train_test_split(
    data_xception, labels_xception, test_size=0.2, random_state=42
)

# Feature extraction using EfficientNetB0
def create_efficientnet_feature_extractor(input_shape):
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Feature extraction using Xception
def create_xception_feature_extractor(input_shape):
    base_model = Xception(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Define the feature extractor model for EfficientNetB0
input_shape_efficientnet = (image_size_efficientnet[0], image_size_efficientnet[1], 3)  # 3 for RGB channels
feature_extractor_efficientnet = create_efficientnet_feature_extractor(input_shape_efficientnet)

# Extract features for EfficientNetB0
x_train_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_train_efficientnet))
x_test_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_test_efficientnet))

# Define the feature extractor model for Xception
input_shape_xception = (image_size_xception[0], image_size_xception[1], 3)  # 3 for RGB channels
feature_extractor_xception = create_xception_feature_extractor(input_shape_xception)

# Extract features for Xception
x_train_features_xception = feature_extractor_xception.predict(np.array(x_train_xception))
x_test_features_xception = feature_extractor_xception.predict(np.array(x_test_xception))

# Reshape the feature arrays before concatenating
x_train_features_efficientnet_reshaped = x_train_features_efficientnet.reshape((x_train_features_efficientnet.shape[0], -1))
x_test_features_efficientnet_reshaped = x_test_features_efficientnet.reshape((x_test_features_efficientnet.shape[0], -1))

x_train_features_xception_reshaped = x_train_features_xception.reshape((x_train_features_xception.shape[0], -1))
x_test_features_xception_reshaped = x_test_features_xception.reshape((x_test_features_xception.shape[0], -1))

# Concatenate the reshaped features from both models
x_train_features_combined = np.concatenate((x_train_features_efficientnet_reshaped, x_train_features_xception_reshaped), axis=1)
x_test_features_combined = np.concatenate((x_test_features_efficientnet_reshaped, x_test_features_xception_reshaped), axis=1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # Enable probability estimates
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier on the combined features
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    # Train the classifier
    classifier.fit(x_train_features_combined, y_train_efficientnet)
    
    # Predictions for accuracy
    y_pred = classifier.predict(x_test_features_combined)
    
    # Accuracy
    accuracy = accuracy_score(y_test_efficientnet, y_pred)
    accuracies.append(accuracy)
    
    # Log loss
    y_pred_proba = classifier.predict_proba(x_test_features_combined)
    log_loss_value = log_loss(y_test_efficientnet, y_pred_proba)
    log_losses.append(log_loss_value)
    
    # Print results
    print(f"{name} accuracy: {accuracy}, log loss: {log_loss_value}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")


19/19 [==============================] - 8s 372ms/step
AdaBoost accuracy: 0.9233333333333333, log loss: 0.6283838979671037
KNN accuracy: 0.97, log loss: 0.1372099395236089
RF accuracy: 0.9633333333333334, log loss: 0.18008833282410772
SVM accuracy: 0.95, log loss: 0.1355488547327469
Softmax accuracy: 0.9883333333333333, log loss: 0.03632919606052738

Average Accuracy: 0.959
Average Log Loss: 0.2235120442216189


In [4]:
#InceptionV3 + EfficientNetB0

import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size_efficientnet = (128, 128)  # EfficientNetB0's required input size
image_size_inception = (128, 128)  # InceptionV3's required input size
batch_size = 32



# Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)

    return data, labels

# Load the EfficientNetB0 dataset
data_efficientnet, labels_efficientnet = load_dataset(original_dataset_path, image_size_efficientnet)

# Split the EfficientNetB0 data into training and testing sets
x_train_efficientnet, x_test_efficientnet, y_train_efficientnet, y_test_efficientnet = train_test_split(
    data_efficientnet, labels_efficientnet, test_size=0.2, random_state=42
)

# Load the InceptionV3 dataset
data_inception, labels_inception = load_dataset(original_dataset_path, image_size_inception)

# Split the InceptionV3 data into training and testing sets
x_train_inception, x_test_inception, y_train_inception, y_test_inception = train_test_split(
    data_inception, labels_inception, test_size=0.2, random_state=42
)

# Feature extraction using EfficientNetB0
def create_efficientnet_feature_extractor(input_shape):
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Feature extraction using InceptionV3
def create_inceptionv3_feature_extractor(input_shape):
    base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Define the feature extractor model for EfficientNetB0
input_shape_efficientnet = (image_size_efficientnet[0], image_size_efficientnet[1], 3)  # 3 for RGB channels
feature_extractor_efficientnet = create_efficientnet_feature_extractor(input_shape_efficientnet)

# Extract features for EfficientNetB0
x_train_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_train_efficientnet))
x_test_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_test_efficientnet))

# Define the feature extractor model for InceptionV3
input_shape_inception = (image_size_inception[0], image_size_inception[1], 3)  # 3 for RGB channels
feature_extractor_inception = create_inceptionv3_feature_extractor(input_shape_inception)

# Extract features for InceptionV3
x_train_features_inception = feature_extractor_inception.predict(np.array(x_train_inception))
x_test_features_inception = feature_extractor_inception.predict(np.array(x_test_inception))

# Reshape the feature arrays before concatenating
x_train_features_efficientnet_reshaped = x_train_features_efficientnet.reshape((x_train_features_efficientnet.shape[0], -1))
x_test_features_efficientnet_reshaped = x_test_features_efficientnet.reshape((x_test_features_efficientnet.shape[0], -1))

x_train_features_inception_reshaped = x_train_features_inception.reshape((x_train_features_inception.shape[0], -1))
x_test_features_inception_reshaped = x_test_features_inception.reshape((x_test_features_inception.shape[0], -1))

# Concatenate the reshaped features from both models
x_train_features_combined = np.concatenate((x_train_features_efficientnet_reshaped, x_train_features_inception_reshaped), axis=1)
x_test_features_combined = np.concatenate((x_test_features_efficientnet_reshaped, x_test_features_inception_reshaped), axis=1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # Enable probability estimates
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier on the combined features
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    # Train the classifier
    classifier.fit(x_train_features_combined, y_train_efficientnet)
    
    # Predictions for accuracy
    y_pred = classifier.predict(x_test_features_combined)
    
    # Accuracy
    accuracy = accuracy_score(y_test_efficientnet, y_pred)
    accuracies.append(accuracy)
    
    # Log loss
    y_pred_proba = classifier.predict_proba(x_test_features_combined)
    log_loss_value = log_loss(y_test_efficientnet, y_pred_proba)
    log_losses.append(log_loss_value)
    
    # Print results
    print(f"{name} accuracy: {accuracy}, log loss: {log_loss_value}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")

19/19 [==============================] - 3s 140ms/step
AdaBoost accuracy: 0.9383333333333334, log loss: 0.625064170248433
KNN accuracy: 0.9633333333333334, log loss: 0.30897467781745463
RF accuracy: 0.945, log loss: 0.2083767015147035
SVM accuracy: 0.96, log loss: 0.09053887534166741
Softmax accuracy: 0.9866666666666667, log loss: 0.059066748479041635

Average Accuracy: 0.9586666666666666
Average Log Loss: 0.2584042346802601


In [2]:
#InceptionV3 + Xception
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import InceptionV3, Xception  # Change here
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size_inception = (128, 128)  # InceptionV3's required input size
image_size_xception = (128, 128)  # Xception's required input size
batch_size = 32

# Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)

    return data, labels

# Load the InceptionV3 dataset
data_inception, labels_inception = load_dataset(original_dataset_path, image_size_inception)

# Split the InceptionV3 data into training and testing sets
x_train_inception, x_test_inception, y_train_inception, y_test_inception = train_test_split(
    data_inception, labels_inception, test_size=0.2, random_state=42
)

# Load the Xception dataset
data_xception, labels_xception = load_dataset(original_dataset_path, image_size_xception)

# Split the Xception data into training and testing sets
x_train_xception, x_test_xception, y_train_xception, y_test_xception = train_test_split(
    data_xception, labels_xception, test_size=0.2, random_state=42
)

# Feature extraction using InceptionV3
def create_inceptionv3_feature_extractor(input_shape):
    base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Feature extraction using Xception
def create_xception_feature_extractor(input_shape):
    base_model = Xception(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Define the feature extractor model for InceptionV3
input_shape_inception = (image_size_inception[0], image_size_inception[1], 3)  # 3 for RGB channels
feature_extractor_inception = create_inceptionv3_feature_extractor(input_shape_inception)

# Extract features for InceptionV3
x_train_features_inception = feature_extractor_inception.predict(np.array(x_train_inception))
x_test_features_inception = feature_extractor_inception.predict(np.array(x_test_inception))

# Define the feature extractor model for Xception
input_shape_xception = (image_size_xception[0], image_size_xception[1], 3)  # 3 for RGB channels
feature_extractor_xception = create_xception_feature_extractor(input_shape_xception)

# Extract features for Xception
x_train_features_xception = feature_extractor_xception.predict(np.array(x_train_xception))
x_test_features_xception = feature_extractor_xception.predict(np.array(x_test_xception))

# Reshape the feature arrays before concatenating
x_train_features_inception_reshaped = x_train_features_inception.reshape((x_train_features_inception.shape[0], -1))
x_test_features_inception_reshaped = x_test_features_inception.reshape((x_test_features_inception.shape[0], -1))

x_train_features_xception_reshaped = x_train_features_xception.reshape((x_train_features_xception.shape[0], -1))
x_test_features_xception_reshaped = x_test_features_xception.reshape((x_test_features_xception.shape[0], -1))

# Concatenate the reshaped features from both models
x_train_features_combined = np.concatenate((x_train_features_inception_reshaped, x_train_features_xception_reshaped), axis=1)
x_test_features_combined = np.concatenate((x_test_features_inception_reshaped, x_test_features_xception_reshaped), axis=1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # Enable probability estimates
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier on the combined features
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    # Train the classifier
    classifier.fit(x_train_features_combined, y_train_inception)
    
    # Predictions for accuracy
    y_pred = classifier.predict(x_test_features_combined)
    
    # Accuracy
    accuracy = accuracy_score(y_test_inception, y_pred)
    accuracies.append(accuracy)
    
    # Log loss
    y_pred_proba = classifier.predict_proba(x_test_features_combined)
    log_loss_value = log_loss(y_test_inception, y_pred_proba)
    log_losses.append(log_loss_value)
    
    # Print results
    print(f"{name} accuracy: {accuracy}, log loss: {log_loss_value}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")


19/19 [==============================] - 7s 334ms/step
AdaBoost accuracy: 0.9466666666666667, log loss: 0.621810179346457
KNN accuracy: 0.9733333333333334, log loss: 0.17411617953197223
RF accuracy: 0.96, log loss: 0.20486897867292206
SVM accuracy: 0.975, log loss: 0.051380443381658175
Softmax accuracy: 0.9883333333333333, log loss: 0.044646459459649186

Average Accuracy: 0.9686666666666668
Average Log Loss: 0.21936444807853173


In [2]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, log_loss
from sklearn.utils import shuffle
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.applications import EfficientNetB0, Xception, InceptionV3
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import joblib

# Define the path to the original dataset
original_dataset_path = 'C:/Users/st282/Brain MRI'

# Data preprocessing and augmentation
image_size_efficientnet = (128, 128)  # EfficientNetB0's required input size
image_size_xception = (128, 128)  # Xception's required input size
image_size_inception = (128, 128)  # InceptionV3's required input size
batch_size = 32

# Function to apply filters to an image
def apply_filters(image):
    # Gaussian Blur and Sobel Edge Detection
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    sobelx = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=1, dy=0, ksize=5)
    sobely = cv2.Sobel(src=blurred, ddepth=cv2.CV_64F, dx=0, dy=1, ksize=5)
    return cv2.magnitude(sobelx, sobely)

# Function to preprocess and augment data
def preprocess_and_augment_data(data, labels, image_size):
    # Resize images
    data = [cv2.resize(img, image_size) for img in data]

    # Data normalization
    data = np.array(data) / 255.0

    return data, labels

# Load and preprocess the dataset
def load_dataset(dataset_path, image_size):
    data = []
    labels = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            if img is not None:  # Check if the image was loaded successfully
                data.append(img)
                labels.append(folder)
            else:
                print(f"Error loading image: {img_path}")

    # Shuffle the data
    data, labels = shuffle(data, labels, random_state=42)

    # Preprocess and augment the loaded data
    data, labels = preprocess_and_augment_data(data, labels, image_size)

    return data, labels

# Load the EfficientNetB0 dataset
data_efficientnet, labels_efficientnet = load_dataset(original_dataset_path, image_size_efficientnet)

# Split the EfficientNetB0 data into training and testing sets
x_train_efficientnet, x_test_efficientnet, y_train_efficientnet, y_test_efficientnet = train_test_split(
    data_efficientnet, labels_efficientnet, test_size=0.2, random_state=42
)

# Load the Xception dataset
data_xception, labels_xception = load_dataset(original_dataset_path, image_size_xception)

# Split the Xception data into training and testing sets
x_train_xception, x_test_xception, y_train_xception, y_test_xception = train_test_split(
    data_xception, labels_xception, test_size=0.2, random_state=42
)

# Load the InceptionV3 dataset
data_inception, labels_inception = load_dataset(original_dataset_path, image_size_inception)

# Split the InceptionV3 data into training and testing sets
x_train_inception, x_test_inception, y_train_inception, y_test_inception = train_test_split(
    data_inception, labels_inception, test_size=0.2, random_state=42
)

# Feature extraction using EfficientNetB0
def create_efficientnet_feature_extractor(input_shape):
    base_model = EfficientNetB0(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Feature extraction using Xception
def create_xception_feature_extractor(input_shape):
    base_model = Xception(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Feature extraction using InceptionV3
def create_inceptionv3_feature_extractor(input_shape):
    base_model = InceptionV3(input_shape=input_shape, include_top=False, weights='imagenet')
    base_model.trainable = False
    return base_model

# Define the feature extractor models
input_shape_efficientnet = (image_size_efficientnet[0], image_size_efficientnet[1], 3)  # 3 for RGB channels
feature_extractor_efficientnet = create_efficientnet_feature_extractor(input_shape_efficientnet)

input_shape_xception = (image_size_xception[0], image_size_xception[1], 3)  # 3 for RGB channels
feature_extractor_xception = create_xception_feature_extractor(input_shape_xception)

input_shape_inception = (image_size_inception[0], image_size_inception[1], 3)  # 3 for RGB channels
feature_extractor_inception = create_inceptionv3_feature_extractor(input_shape_inception)

# Extract features
x_train_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_train_efficientnet))
x_test_features_efficientnet = feature_extractor_efficientnet.predict(np.array(x_test_efficientnet))

x_train_features_xception = feature_extractor_xception.predict(np.array(x_train_xception))
x_test_features_xception = feature_extractor_xception.predict(np.array(x_test_xception))

x_train_features_inception = feature_extractor_inception.predict(np.array(x_train_inception))
x_test_features_inception = feature_extractor_inception.predict(np.array(x_test_inception))

# Reshape the feature arrays before concatenating
x_train_features_efficientnet_reshaped = x_train_features_efficientnet.reshape((x_train_features_efficientnet.shape[0], -1))
x_test_features_efficientnet_reshaped = x_test_features_efficientnet.reshape((x_test_features_efficientnet.shape[0], -1))

x_train_features_xception_reshaped = x_train_features_xception.reshape((x_train_features_xception.shape[0], -1))
x_test_features_xception_reshaped = x_test_features_xception.reshape((x_test_features_xception.shape[0], -1))

x_train_features_inception_reshaped = x_train_features_inception.reshape((x_train_features_inception.shape[0], -1))
x_test_features_inception_reshaped = x_test_features_inception.reshape((x_test_features_inception.shape[0], -1))

# Concatenate the reshaped features from all models
x_train_features_combined = np.concatenate((x_train_features_efficientnet_reshaped,
                                             x_train_features_xception_reshaped,
                                             x_train_features_inception_reshaped), axis=1)

x_test_features_combined = np.concatenate((x_test_features_efficientnet_reshaped,
                                            x_test_features_xception_reshaped,
                                            x_test_features_inception_reshaped), axis=1)

# Initialize classifiers
classifiers = {
    "AdaBoost": AdaBoostClassifier(),
    "KNN": KNeighborsClassifier(),
    "RF": RandomForestClassifier(),
    "SVM": SVC(probability=True),  # Enable probability estimates
    "Softmax": LogisticRegression(multi_class='multinomial', max_iter=1000)
}

# Train and evaluate each classifier on the combined features
accuracies = []
log_losses = []

for name, classifier in classifiers.items():
    # Train the classifier
    classifier.fit(x_train_features_combined, y_train_efficientnet)
    
    # Predictions for accuracy
    y_pred = classifier.predict(x_test_features_combined)
    
    # Accuracy
    accuracy = accuracy_score(y_test_efficientnet, y_pred)
    accuracies.append(accuracy)
    
    # Log loss
    y_pred_proba = classifier.predict_proba(x_test_features_combined)
    log_loss_value = log_loss(y_test_efficientnet, y_pred_proba)
    log_losses.append(log_loss_value)
    
    # Print results
    print(f"{name} accuracy: {accuracy}, log loss: {log_loss_value}")

# Calculate average accuracy
average_accuracy = np.mean(accuracies)
print(f"\nAverage Accuracy: {average_accuracy}")

# Calculate average log loss
average_log_loss = np.mean(log_losses)
print(f"Average Log Loss: {average_log_loss}")


# Save the trained ensemble model
ensemble_model_path = 'ensemble_model.joblib'
joblib.dump(classifiers, ensemble_model_path)



19/19 [==============================] - 4s 146ms/step
AdaBoost accuracy: 0.9466666666666667, log loss: 0.6189339508948936
KNN accuracy: 0.9733333333333334, log loss: 0.17411617953197223
RF accuracy: 0.9633333333333334, log loss: 0.17776344745566736
SVM accuracy: 0.975, log loss: 0.061273398733576144
Softmax accuracy: 0.9866666666666667, log loss: 0.04439643648403736

Average Accuracy: 0.969
Average Log Loss: 0.21529668262002932


['ensemble_model.joblib']

In [25]:
import cv2
import numpy as np
from tensorflow import keras
import joblib

# Load the ensemble model
loaded_classifiers = joblib.load(ensemble_model_path)

# Function to predict using the loaded model
def predict_image_class(image_path, loaded_model):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, image_size_efficientnet) / 255.0
        img = img.reshape(1, image_size_efficientnet[0], image_size_efficientnet[1], 3)  # Reshape for InceptionV3

        features_inception = feature_extractor_inception.predict(img)
        features_efficientnet = feature_extractor_efficientnet.predict(img)
        features_xception = feature_extractor_xception.predict(img)

        features_inception = features_inception.reshape(1, -1)
        features_efficientnet = features_efficientnet.reshape(1, -1)
        features_xception = features_xception.reshape(1, -1)

        combined_features = np.concatenate((features_efficientnet, features_xception, features_inception), axis=1)

        # Iterate over each classifier in the ensemble and predict
        results = {}
        for name, classifier in loaded_model.items():
            result = classifier.predict(combined_features)[0]
            results[name] = result

        return results
    else:
        return None

# Counting 'Yes' and 'No' predictions and returning the majority
def majority_vote(results):
    yes_count = sum([1 for prediction in results.values() if prediction.lower() == 'yes'])
    no_count = sum([1 for prediction in results.values() if prediction.lower() == 'no'])

    return 'Yes' if yes_count > no_count else 'No'

# Example usage
image_path_to_predict = 'C:/uu.jpg'
result = predict_image_class(image_path_to_predict, loaded_classifiers)

if result is not None:
    print("Predictions by individual classifiers :")
    for name, prediction in result.items():
        print(f"{name}: {prediction}")

    # Get the majority vote
    majority_prediction = majority_vote(result)
    print(f"Majority Prediction: {majority_prediction}")
else:
    print("Error loading the image.")

1/1 [==============================] - 0s 43ms/step
Predictions by individual classifiers :
AdaBoost: no
KNN: yes
RF: no
SVM: no
Softmax: no
Majority Prediction: No
